In [2]:
import cobra
import libsbml
import riptide
from cobra import Model, Reaction, Metabolite
import csv
import pandas as pd
import copy

In [3]:
ios2164_orig = cobra.io.read_sbml_model("../model/iOS2164_rewritten.xml")
rnaseq = pd.read_csv('../data/KitaakeLCM_rawcounts.csv')
#According to the  tool's author, the best way to merge is simply to generate 

In [4]:
mcell_exp = riptide.read_transcription_file("../data/m_abundance.csv", sep = ",", header = True)
bscell_exp = riptide.read_transcription_file("../data/bs_abundance.csv", sep = ",", header = True)
vcell_exp = riptide.read_transcription_file("../data/v_abundance.csv", sep = ",", header = True)

In [24]:
ios2164_v = copy.deepcopy(ios2164_orig)

for met in ios2164_v.metabolites:
    if met.compartment != "e": #Extracellular, defines boundary reactions
        met.id = str(met.id) + '2'
        met.compartment = str(met.compartment) + '2'
    
for rxn in ios2164_v.reactions:
    rxn.id = str(rxn.id)  + '_V'
    if "EX_" in rxn.id: #Rename media reactions
        rxn.name = "V " + str(rxn.name)


ios2164_v.repair()


In [37]:
read = pd.read_csv('../misc/photo_medium.csv')

#Only get V cell media
photo_medium = dict(read.values)

v_medium = dict()
for items in photo_medium:
    if "_V" in items:
        v_medium[items] = photo_medium[items]
        
v_copy = copy.deepcopy(v_medium)


In [38]:
v_medium

{'EX_co2_LPAREN_e_RPAREN__V': 1000,
 'EX_h2o_LPAREN_e_RPAREN__V': 1000,
 'EX_h_LPAREN_e_RPAREN__V': 1000,
 'EX_no3_LPAREN_e_RPAREN__V': 1000,
 'EX_o2_LPAREN_e_RPAREN__V': 1000,
 'EX_pi_LPAREN_e_RPAREN__V': 1000,
 'EX_sucr_LPAREN_e_RPAREN__V': 0,
 'EX_fru_DASH_B_LPAREN_e_RPAREN__V': 0,
 'EX_glc_DASH_A_LPAREN_e_RPAREN__V': 0,
 'EX_so4_LPAREN_e_RPAREN__V': 1000,
 'EX_hco3_LPAREN_e_RPAREN__V': 0,
 'EX_so3_LPAREN_e_RPAREN__V': 1000,
 'EX_h2s_LPAREN_e_RPAREN__V': 0,
 'EX_nh4_LPAREN_e_RPAREN__V': 0,
 'EX_asn_DASH_L_LPAREN_e_RPAREN__V': 0,
 'EX_gln_DASH_L_LPAREN_e_RPAREN__V': 0,
 'EX_etoh_LPAREN_e_RPAREN__V': 0,
 'EX_ac_LPAREN_e_RPAREN__V': 0,
 'EX_lac_DASH_L_LPAREN_e_RPAREN__V': 0,
 'EX_tsul_LPAREN_e_RPAREN__V': 0,
 'EX_fe2_LPAREN_e_RPAREN__V': 1000,
 'EX_fe3_LPAREN_e_RPAREN__V': 0,
 'EX_mg2_LPAREN_e_RPAREN__V': 1000,
 'EX_ala_DASH_L_LPAREN_e_RPAREN__V': 0,
 'EX_arg_DASH_L_LPAREN_e_RPAREN__V': 0,
 'EX_asp_DASH_L_LPAREN_e_RPAREN__V': 0,
 'EX_cys_DASH_L_LPAREN_e_RPAREN__V': 0,
 'EX_glu_DASH_L_L

In [39]:
v_medium = copy.copy(v_copy)
ios2164_v.medium = v_medium
vcell_rip  = riptide.contextualize(ios2164_v, transcriptome = vcell_exp, samples= 1000)

#Apparently unconstraining the export of sucrose doesn't actually do anything.


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 452 from 2444 (81.51% change)
Metabolites pruned to 469 from 1999 (76.54% change)
Flux through the objective DECREASED to ~9.64 from ~11.14 (13.46% change)
Context-specific metabolism correlates with transcriptome (r=0.181, p<0.001 *)

RIPTiDe completed in 35 seconds



In [41]:
for rxns in ios2164_v.exchanges:
    print(rxns.name, rxns.bounds)

V CO2 exchange (-1000, 0.0)
V H2O exchange (-1000, 1000.0)
V H+ exchange (-1000, 1000.0)
V nitrate exchange (-1000, 0.0)
V O2 exchange (-1000, 1000.0)
V Phosphate exchange (-1000, 1000.0)
V Sucrose exchange (0, 0.0)
V Fructose exchange (0, 0.0)
V Glucose exchange (0, 0.0)
V Sulfate exchange (-1000, 1000.0)
V Bicarbonate exchange (0, 0.0)
V Sulfite exchange (-1000, 0.0)
V Hydrogen sulfide exchange (0, 0.0)
V Ammonia exchange (0, 0.0)
V L-Asparagine exchange (0, 1000.0)
V L-Glutamine exchange (0, 1000.0)
V Ethanol exchange (0, 1000.0)
V Acetate exchange (0, 1000.0)
V L-Lactate exchange (0, 1000.0)
V Thiosulfate exchange (0, 0.0)
V Fe2+ exchange (-1000, 1000.0)
V Fe3+ exchange (0, 0.0)
V Magnesium exchange (-1000, 1000.0)
V L-Alanine exchange (0, 1000.0)
V L-Arginine exchange (0, 1000.0)
V L-Aspartate exchange (0, 1000.0)
V L-Cysteine exchange (0, 1000.0)
V L-Glutamate exchange (0, 1000.0)
V Glycine exchange (0, 1000.0)
V L-Histidine exchange (0, 1000.0)
V L-Isoleucine exchange (0, 1000.0

In [50]:
#What if I unconstrained CO2 flux thru the BS cell? Would the context specific cells be okay?
v_medium = copy.copy(v_copy)

bs_medium['EX_co2_LPAREN_e_RPAREN__V'] = 1000
ios2164_v.medium = v_medium

vcell_rip  = riptide.contextualize(ios2164_v, transcriptome = vcell_exp, samples= 1000)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 452 from 2444 (81.51% change)
Metabolites pruned to 469 from 1999 (76.54% change)
Flux through the objective DECREASED to ~9.64 from ~11.14 (13.46% change)
Context-specific metabolism correlates with transcriptome (r=0.175, p<0.001 *)

RIPTiDe completed in 37 seconds



In [51]:
#With Sucrose import, apparently it lowers the correlation value as the cell can obtain
#Nutrients 
v_medium = copy.copy(v_medium)

v_medium['EX_sucr_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_co2_LPAREN_e_RPAREN__V'] = 1000
ios2164_v.medium = v_medium

vcell_rip  = riptide.contextualize(ios2164_v, transcriptome = vcell_exp, samples= 1000)

ios2164_v.medium

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 454 from 2444 (81.42% change)
Metabolites pruned to 475 from 1999 (76.24% change)
Flux through the objective DECREASED to ~31.5 from ~38.12 (17.37% change)
Context-specific metabolism correlates with transcriptome (r=0.142, p=0.002 *)

RIPTiDe completed in 38 seconds



{'EX_co2_LPAREN_e_RPAREN__V': 1000,
 'EX_h2o_LPAREN_e_RPAREN__V': 1000,
 'EX_h_LPAREN_e_RPAREN__V': 1000,
 'EX_no3_LPAREN_e_RPAREN__V': 1000,
 'EX_o2_LPAREN_e_RPAREN__V': 1000,
 'EX_pi_LPAREN_e_RPAREN__V': 1000,
 'EX_sucr_LPAREN_e_RPAREN__V': 1000,
 'EX_so4_LPAREN_e_RPAREN__V': 1000,
 'EX_so3_LPAREN_e_RPAREN__V': 1000,
 'EX_fe2_LPAREN_e_RPAREN__V': 1000,
 'EX_mg2_LPAREN_e_RPAREN__V': 1000,
 'EX_photonVis_LPAREN_e_RPAREN__V': 1000}

In [52]:
#Gas exchange is allowed, but photoautotrophic conditions -- no sucrose import or anything

v_medium = copy.copy(v_copy)

v_medium['EX_o2_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_co2_LPAREN_e_RPAREN__V'] = 1000
ios2164_v.medium = v_medium

vcell_rip  = riptide.contextualize(ios2164_v, transcriptome = vcell_exp, samples= 1000)

ios2164_v.medium

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 450 from 2444 (81.59% change)
Metabolites pruned to 466 from 1999 (76.69% change)
Flux through the objective DECREASED to ~9.64 from ~11.14 (13.46% change)
Context-specific metabolism correlates with transcriptome (r=0.175, p<0.001 *)

RIPTiDe completed in 37 seconds



{'EX_co2_LPAREN_e_RPAREN__V': 1000,
 'EX_h2o_LPAREN_e_RPAREN__V': 1000,
 'EX_h_LPAREN_e_RPAREN__V': 1000,
 'EX_no3_LPAREN_e_RPAREN__V': 1000,
 'EX_o2_LPAREN_e_RPAREN__V': 1000,
 'EX_pi_LPAREN_e_RPAREN__V': 1000,
 'EX_so4_LPAREN_e_RPAREN__V': 1000,
 'EX_so3_LPAREN_e_RPAREN__V': 1000,
 'EX_fe2_LPAREN_e_RPAREN__V': 1000,
 'EX_mg2_LPAREN_e_RPAREN__V': 1000,
 'EX_photonVis_LPAREN_e_RPAREN__V': 1000}

In [61]:
#Try adding some ammonia into the medium

v_medium = copy.copy(v_copy)

v_medium['EX_o2_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_co2_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_nh4_LPAREN_e_RPAREN__V'] = 1000
ios2164_v.medium = v_medium

vcell_rip  = riptide.contextualize(ios2164_v, transcriptome = vcell_exp, samples= 1000)

ios2164_v.medium

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 448 from 2444 (81.67% change)
Metabolites pruned to 463 from 1999 (76.84% change)
Flux through the objective DECREASED to ~9.17 from ~11.18 (17.98% change)
Context-specific metabolism correlates with transcriptome (r=0.183, p<0.001 *)

RIPTiDe completed in 37 seconds



{'EX_co2_LPAREN_e_RPAREN__V': 1000,
 'EX_h2o_LPAREN_e_RPAREN__V': 1000,
 'EX_h_LPAREN_e_RPAREN__V': 1000,
 'EX_no3_LPAREN_e_RPAREN__V': 1000,
 'EX_o2_LPAREN_e_RPAREN__V': 1000,
 'EX_pi_LPAREN_e_RPAREN__V': 1000,
 'EX_so4_LPAREN_e_RPAREN__V': 1000,
 'EX_so3_LPAREN_e_RPAREN__V': 1000,
 'EX_nh4_LPAREN_e_RPAREN__V': 1000,
 'EX_fe2_LPAREN_e_RPAREN__V': 1000,
 'EX_mg2_LPAREN_e_RPAREN__V': 1000,
 'EX_photonVis_LPAREN_e_RPAREN__V': 1000}

In [47]:
#With a single AA, test lang haha
v_medium = copy.copy(v_copy)

v_medium['EX_o2_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_co2_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_nh4_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_ala_DASH_L_LPAREN_e_RPAREN__V'] = 1000
ios2164_v.medium = v_medium

vcell_rip  = riptide.contextualize(ios2164_v, transcriptome = vcell_exp, samples= 1000)

ios2164_v.medium


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 441 from 2444 (81.96% change)
Metabolites pruned to 463 from 1999 (76.84% change)
Flux through the objective DECREASED to ~10.07 from ~12.93 (22.12% change)
Context-specific metabolism correlates with transcriptome (r=0.227, p<0.001 *)

RIPTiDe completed in 37 seconds



{'EX_co2_LPAREN_e_RPAREN__V': 1000,
 'EX_h2o_LPAREN_e_RPAREN__V': 1000,
 'EX_h_LPAREN_e_RPAREN__V': 1000,
 'EX_no3_LPAREN_e_RPAREN__V': 1000,
 'EX_o2_LPAREN_e_RPAREN__V': 1000,
 'EX_pi_LPAREN_e_RPAREN__V': 1000,
 'EX_so4_LPAREN_e_RPAREN__V': 1000,
 'EX_so3_LPAREN_e_RPAREN__V': 1000,
 'EX_nh4_LPAREN_e_RPAREN__V': 1000,
 'EX_fe2_LPAREN_e_RPAREN__V': 1000,
 'EX_mg2_LPAREN_e_RPAREN__V': 1000,
 'EX_ala_DASH_L_LPAREN_e_RPAREN__V': 1000,
 'EX_photonVis_LPAREN_e_RPAREN__V': 1000}

In [68]:
#With a single AA, test lang haha
v_medium = copy.copy(v_copy)

v_medium['EX_o2_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_co2_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_nh4_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_sucr_LPAREN_e_RPAREN__V'] = 2000
v_medium['EX_arg_DASH_L_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_photonVis_LPAREN_e_RPAREN__V'] = 0
ios2164_v.medium = v_medium

vcell_rip  = riptide.contextualize(ios2164_v, transcriptome = vcell_exp, samples= 1000)

ios2164_v.medium

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 429 from 2444 (82.45% change)
Metabolites pruned to 437 from 1999 (78.14% change)
Flux through the objective DECREASED to ~34.51 from ~42.91 (19.58% change)
Context-specific metabolism correlates with transcriptome (r=0.18, p<0.001 *)

RIPTiDe completed in 35 seconds



{'EX_co2_LPAREN_e_RPAREN__V': 1000,
 'EX_h2o_LPAREN_e_RPAREN__V': 1000,
 'EX_h_LPAREN_e_RPAREN__V': 1000,
 'EX_no3_LPAREN_e_RPAREN__V': 1000,
 'EX_o2_LPAREN_e_RPAREN__V': 1000,
 'EX_pi_LPAREN_e_RPAREN__V': 1000,
 'EX_sucr_LPAREN_e_RPAREN__V': 2000,
 'EX_so4_LPAREN_e_RPAREN__V': 1000,
 'EX_so3_LPAREN_e_RPAREN__V': 1000,
 'EX_nh4_LPAREN_e_RPAREN__V': 1000,
 'EX_fe2_LPAREN_e_RPAREN__V': 1000,
 'EX_mg2_LPAREN_e_RPAREN__V': 1000,
 'EX_arg_DASH_L_LPAREN_e_RPAREN__V': 1000}

In [60]:
#With a single AA, test lang haha
v_medium = copy.copy(v_copy)

v_medium['EX_o2_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_co2_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_nh4_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_photonVis_LPAREN_e_RPAREN__V'] = 0
v_medium['EX_ala_DASH_L_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_arg_DASH_L_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_asn_DASH_L_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_asp_DASH_L_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_his_DASH_L_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_ile_DASH_L_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_leu_DASH_L_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_lys_DASH_L_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_ser_DASH_L_LPAREN_e_RPAREN__V'] = 1000
ios2164_v.medium = v_medium

vcell_rip  = riptide.contextualize(ios2164_v, transcriptome = vcell_exp, samples= 1000)

ios2164_v.medium

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 416 from 2444 (82.98% change)
Metabolites pruned to 431 from 1999 (78.44% change)
Flux through the objective DECREASED to ~16.12 from ~19.18 (15.95% change)
Context-specific metabolism correlates with transcriptome (r=0.117, p=0.017 *)

RIPTiDe completed in 37 seconds



{'EX_co2_LPAREN_e_RPAREN__V': 1000,
 'EX_h2o_LPAREN_e_RPAREN__V': 1000,
 'EX_h_LPAREN_e_RPAREN__V': 1000,
 'EX_no3_LPAREN_e_RPAREN__V': 1000,
 'EX_o2_LPAREN_e_RPAREN__V': 1000,
 'EX_pi_LPAREN_e_RPAREN__V': 1000,
 'EX_so4_LPAREN_e_RPAREN__V': 1000,
 'EX_so3_LPAREN_e_RPAREN__V': 1000,
 'EX_nh4_LPAREN_e_RPAREN__V': 1000,
 'EX_asn_DASH_L_LPAREN_e_RPAREN__V': 1000,
 'EX_fe2_LPAREN_e_RPAREN__V': 1000,
 'EX_mg2_LPAREN_e_RPAREN__V': 1000,
 'EX_ala_DASH_L_LPAREN_e_RPAREN__V': 1000,
 'EX_arg_DASH_L_LPAREN_e_RPAREN__V': 1000,
 'EX_asp_DASH_L_LPAREN_e_RPAREN__V': 1000,
 'EX_his_DASH_L_LPAREN_e_RPAREN__V': 1000,
 'EX_ile_DASH_L_LPAREN_e_RPAREN__V': 1000,
 'EX_leu_DASH_L_LPAREN_e_RPAREN__V': 1000,
 'EX_lys_DASH_L_LPAREN_e_RPAREN__V': 1000,
 'EX_ser_DASH_L_LPAREN_e_RPAREN__V': 1000}

In [56]:
#Heterotrophic conditions, try lang
v_medium = copy.copy(v_copy)

v_medium['EX_o2_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_co2_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_nh4_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_sucr_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_photonVis_LPAREN_e_RPAREN__V'] = 0
ios2164_v.medium = v_medium

vcell_rip  = riptide.contextualize(ios2164_v, transcriptome = vcell_exp, samples= 1000)

ios2164_v.medium

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 434 from 2444 (82.24% change)
Metabolites pruned to 444 from 1999 (77.79% change)
Flux through the objective DECREASED to ~33.22 from ~39.77 (16.47% change)
Context-specific metabolism correlates with transcriptome (r=0.173, p<0.001 *)

RIPTiDe completed in 37 seconds



{'EX_co2_LPAREN_e_RPAREN__V': 1000,
 'EX_h2o_LPAREN_e_RPAREN__V': 1000,
 'EX_h_LPAREN_e_RPAREN__V': 1000,
 'EX_no3_LPAREN_e_RPAREN__V': 1000,
 'EX_o2_LPAREN_e_RPAREN__V': 1000,
 'EX_pi_LPAREN_e_RPAREN__V': 1000,
 'EX_sucr_LPAREN_e_RPAREN__V': 1000,
 'EX_so4_LPAREN_e_RPAREN__V': 1000,
 'EX_so3_LPAREN_e_RPAREN__V': 1000,
 'EX_nh4_LPAREN_e_RPAREN__V': 1000,
 'EX_fe2_LPAREN_e_RPAREN__V': 1000,
 'EX_mg2_LPAREN_e_RPAREN__V': 1000}

In [57]:
#Heterotrophic conditions, try lang
v_medium = copy.copy(v_copy)

v_medium['EX_o2_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_co2_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_nh4_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_fe3_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_photonVis_LPAREN_e_RPAREN__V'] = 0
ios2164_v.medium = v_medium

vcell_rip  = riptide.contextualize(ios2164_v, transcriptome = vcell_exp, samples= 1000)

ios2164_v.medium

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 425 from 2444 (82.61% change)
Metabolites pruned to 447 from 1999 (77.64% change)
Flux through the objective DECREASED to ~7.57 from ~8.62 (12.18% change)
Context-specific metabolism correlates with transcriptome (r=0.214, p<0.001 *)

RIPTiDe completed in 38 seconds



{'EX_co2_LPAREN_e_RPAREN__V': 1000,
 'EX_h2o_LPAREN_e_RPAREN__V': 1000,
 'EX_h_LPAREN_e_RPAREN__V': 1000,
 'EX_no3_LPAREN_e_RPAREN__V': 1000,
 'EX_o2_LPAREN_e_RPAREN__V': 1000,
 'EX_pi_LPAREN_e_RPAREN__V': 1000,
 'EX_so4_LPAREN_e_RPAREN__V': 1000,
 'EX_so3_LPAREN_e_RPAREN__V': 1000,
 'EX_nh4_LPAREN_e_RPAREN__V': 1000,
 'EX_fe2_LPAREN_e_RPAREN__V': 1000,
 'EX_fe3_LPAREN_e_RPAREN__V': 1000,
 'EX_mg2_LPAREN_e_RPAREN__V': 1000}

In [59]:
#Heterotrophic conditions, try lang
v_medium = copy.copy(v_copy)

v_medium['EX_o2_LPAREN_e_RPAREN__V'] = 0
v_medium['EX_co2_LPAREN_e_RPAREN__V'] = 0
v_medium['EX_nh4_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_fe3_LPAREN_e_RPAREN__V'] = 1000
v_medium['EX_photonVis_LPAREN_e_RPAREN__V'] = 0
ios2164_v.medium = v_medium

vcell_rip  = riptide.contextualize(ios2164_v, transcriptome = vcell_exp, samples= 1000)

ios2164_v.medium


Initializing model and integrating transcriptomic data...


/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 464 from 2444 (81.01% change)
Metabolites pruned to 487 from 1999 (75.64% change)
Flux through the objective DECREASED to ~0.39 from ~0.4 (2.5% change)
Context-specific metabolism correlates with transcriptome (r=0.123, p=0.008 *)

RIPTiDe completed in 35 seconds



{'EX_h2o_LPAREN_e_RPAREN__V': 1000,
 'EX_h_LPAREN_e_RPAREN__V': 1000,
 'EX_no3_LPAREN_e_RPAREN__V': 1000,
 'EX_pi_LPAREN_e_RPAREN__V': 1000,
 'EX_so4_LPAREN_e_RPAREN__V': 1000,
 'EX_so3_LPAREN_e_RPAREN__V': 1000,
 'EX_nh4_LPAREN_e_RPAREN__V': 1000,
 'EX_fe2_LPAREN_e_RPAREN__V': 1000,
 'EX_fe3_LPAREN_e_RPAREN__V': 1000,
 'EX_mg2_LPAREN_e_RPAREN__V': 1000}

Benchmarking results for BS Cells:

No gas exchange, no photon flux: r=0.123, p=0.008
With Gas exchange, no photon flux: r=0.214, p<0.001 
W/ gas exchange, sucrose import, no photon flux (Hetero conditions): r=0.173, p<0.001 
w/ AA import, gas exchange, no : r=0.117, p=0.017

Notes:
Setting up monosaccharide export didn't do much in terms of correlation results.
Okay, select the medium with NH4 + Gas exchange instead for max correlation////